In [1]:
%pylab inline
import numpy as np
from sklearn.datasets import make_classification
num_egs = 5
num_features = 1
num_classifiers = 3

X = np.zeros((num_egs,num_features))
labels = np.zeros(num_egs)
X[0][0]=10; labels[0] = 1;
X[1][0]=30; labels[1] = 1;
X[2][0]=40; labels[2] = -1;
X[3][0]=60; labels[3] = -1;
X[4][0]=90; labels[4] = 1;

# X[0][1]=40; 
# X[1][1]=50; 
# X[2][1]=40; 
# X[3][1]=50; 
# X[4][1]=40; 

# X[0][0] = -1; labels[0] = -1;
# X[1][0] = 0; labels[1] = 1;
# X[2][0] = 1; labels[2] = -1;


Populating the interactive namespace from numpy and matplotlib


In [8]:
import argparse

args = parser.parse_args()

parser = argparse.ArgumentParser(description='Create Dataset.')
parser.add_argument('n', metavar='N', type=int,
                    help='num examples')
parser.add_argument('m', metavar='M', type=int,
                    help='num fetures'


x,y = datasets.make_classification(n_samples=1000, n_features=200)
y[y==0]=-1
pd.DataFrame(x).to_csv('data/{}_{}_data.csv'.format(args.n, argms.m),index=False,header=False)
pd.DataFrame(y).to_csv('data/{}_{}_label.csv'.format(args.n, argms.m),index=False,header=False)


In [27]:
# C++ like data structure
class Decision_Function():
    def __init__(self, error, direction, split_val, feature_index = -99, alpha=1):
        self.error = error
        self.direction = direction
        self.split_val = split_val
        self.feature_index = feature_index
        self.alpha = alpha


class DecisionStump():
    def __init__(self, criteria='gini'):
        self.criteria = criteria

#     def get_ginni_curr_feature_val(self, labels, weights, feature_vals, split_value):
#         left = []
#         right = []
#         leftweight = []
#         rightweight = []
#         gini = 0
#         for i in range(self.num_egs):
#             if (feature_vals[i] <= split_value):
#                 left.append(labels[i])
#                 leftweight.append(weights[i])
#             else:
#                 right.append(labels[i])
#                 rightweight.append(weights[i])
#         for group, weight in ([(left, leftweight), (right, rightweight)]):
#             size = float(len(group))
#             wt_of_group = sum([i for i in weight])
#             if size == 0:
#                 continue
#             score = 0.0
#             p1 = 0
#             p2 = 0
#             for i, label in enumerate(group):
#                 if label == 1:
#                     p1 += weight[i] * 1/wt_of_group
#                 else:
#                     p2 += weight[i] * 1/wt_of_group
#             score = p1 * p1 + p2 * p2 
#             gini += (1.0 - score) * wt_of_group
#         return gini
        
    def get_error_curr_feature_val(self, labels, weights, feature_vals, split_value):
        curr_error = 0
        for i in range(self.num_egs):
            # x[i] < c should be -1
            if  ((feature_vals[i] <= split_value) and (labels[i]!=-1)) or \
                ((feature_vals[i] > split_value) and (labels[i]!=1)):
                curr_error += weights[i];
            
        sol = Decision_Function(curr_error, 1, split_value)
#         gini1 = self.get_ginni_curr_feature_val(labels, weights, feature_vals, split_value)
#         print("Error : {:.2f} \t Gini : {:.2f}".format(curr_error, self.get_ginni_curr_feature_val(labels, weights, feature_vals, split_value)))
        if curr_error > 0.5:
            sol.direction = -1
            sol.error = 1 - curr_error
        return sol
        
    
    def get_error_curr_feature(self, labels, weights, feature_vals, feature_split_values):
        best_decision_function_curr_feature = Decision_Function(float('inf'), 1, float('inf'))
        
        for i in range(len(feature_split_values)):
            curr_decision_function = self.get_error_curr_feature_val(labels, weights, feature_vals, feature_split_values[i])
            
            if curr_decision_function.error < best_decision_function_curr_feature.error:
                best_decision_function_curr_feature = curr_decision_function
            
        return best_decision_function_curr_feature
    
    def fit(self, labels, X_T, weights, feature_split_values):
        
        self.num_features = len(X_T)
        self.num_egs = len(X_T[0])
        
        best_decision_function = Decision_Function(float('inf'), 1, float('inf'))
        
        for i in range(self.num_features):
            current_feature_stump = self.get_error_curr_feature(labels, weights, X_T[i], 
                                                            feature_split_values[i])
            if current_feature_stump.error < best_decision_function.error:
                best_decision_function = current_feature_stump
                best_decision_function.feature_index = i
        return best_decision_function

class AdaBoost():
    def __init__(self, num_classifiers=3):
        self.num_classifiers = num_classifiers
        self.classifiers = None
    def get_feature_split_vals(self, X_T):
        '''
            Args:
                x : dataset
                
            Returns:
                feature_midpoints : For each feature returns a sorted list of mid points 
                                    of every two consecutive unique features
            
            Complexity:
               (Old) :  M + N * M + M * NlogN + M * N
               (New) : M * N log N + M * N
        '''
        
        X_copy = []     
        feature_midpoints = []

        # M * n log * n
        for i in range(self.num_features):
            X_copy.append(sorted(X_T[i]))
        
        # Get mid points of sorted unique values M * N
        
        for i in range(self.num_features):
            current = []
            j = 0
            current.append(X_copy[i][0] - 1)
            while (j < self.num_egs - 1):
                k = j + 1
                while(X_copy[i][j] == X_copy[i][k] and k < self.num_egs - 1):
                    k += 1
                midpoint = (X_copy[i][j] + X_copy[i][k])/2
                current.append(midpoint)
                j = k
            current.append(X_copy[i][self.num_egs - 1] + 1)
            feature_midpoints.append(current)
        return feature_midpoints
    
    def update_weights(self, classifier, X_T, weights, labels):

        Z = 0
        feature_index = classifier.feature_index
        
        for i in range(self.num_egs):
            if classifier.direction == 1:
                if X_T[feature_index][i] <= classifier.split_val:
                    weights[i] *= np.exp(classifier.alpha * labels[i])
                else:
                    weights[i] *= np.exp(-classifier.alpha * labels[i])
            elif classifier.direction == -1:
                if X_T[feature_index][i] >= classifier.split_val:
                    weights[i] *= np.exp(classifier.alpha * labels[i])
                else:
                    weights[i] *= np.exp(-classifier.alpha * labels[i])
            Z += weights[i]

        return weights/Z

    
    def fit_weak_classifers(self, labels, X_T, feature_split_values):
        weights = np.ones(self.num_egs)/(self.num_egs)
        classifiers = []
        for t in range(self.num_classifiers): 
            clf = DecisionStump()
            classifiers.append(clf.fit(labels, X_T, weights, feature_split_values))
            alpha = np.log((1 - classifiers[-1].error)/classifiers[-1].error) * 0.5
            classifiers[-1].alpha = alpha
            weights = self.update_weights(classifiers[-1], X_T, weights, labels) 

#             print("t : {} \t alpha : {:.2f} \t ft_index : {} \t direction : {:2} \t split_value : {} \t error : {:.2f}".format(
#             t, classifiers[-1].alpha, classifiers[-1].feature_index, classifiers[-1].direction, classifiers[-1].split_val, classifiers[-1].error))
            
        return classifiers

    def fit(self, X, labels):
        self.num_egs = len(X)
        self.num_features = len(X[0])
        X_T = X.T
        feature_split_values = self.get_feature_split_vals(X_T)
        classifiers = self.fit_weak_classifers(labels, X_T,feature_split_values )
        self.classifiers = classifiers
    def predict(self, X):
        a = 0
        predicted = np.zeros(len(X))

        for num, eg in enumerate(X):
            a = 0
            for clf in self.classifiers:
                if clf.direction == 1:
                    if eg[int(clf.feature_index)] <= clf.split_val:
                        a += clf.alpha * -1
                    else:
                        a += clf.alpha
                else:
                    if eg[int(clf.feature_index)] >= clf.split_val:
                        a += clf.alpha * -1
                    else:
                        a += clf.alpha
            predicted[num] = np.sign(a)
        return predicted
#         for t in range(self.num_classifiers):
            

In [28]:
clf = AdaBoost(num_classifiers=3)
%timeit clf.fit(X,labels)
clf.predict(X)

169 µs ± 18.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


array([ 1.,  1., -1., -1.,  1.])

In [ ]:
clf = AdaBoost(num_classifiers=10)
clf.fit(X,labels)